# Human-mapping

In [ ]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm_notebook as tq

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

### Mapping family name

In [ ]:
file_askp = pd.read_csv('askp.csv', sep=';', encoding = 'utf-8', error_bad_lines=False, low_memory=False)

file_sap = pd.read_csv('sap.csv', sep=';', encoding = 'utf-8', error_bad_lines=False, low_memory=False)

In [ ]:
askp = file_askp.values
sap = file_sap.values

In [ ]:
def preproc(s):
    """
    Function to preprocessing text data
    """
    s = s.lower().replace('ё', 'е')
    s = re.sub(r'\d+', '', s)
    s = re.sub(r'\W', ' ', s)
    s = tokenizer.tokenize(s)
    return s

In [ ]:
askp = [preproc(i[0]) for i in askp]
sap = [preproc(i[0]) for i in sap]

In [ ]:
def distance(a, b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a, b = b, a
        n, m = m, n
    current_row = range(n+1) # Keep current and previous row, not entire matrix
    for i in range(1, m+1):
        previous_row, current_row = current_row, [i]+[0]*n
        for j in range(1,n+1):
            add, delete, change = previous_row[j]+1, current_row[j-1]+1, previous_row[j-1]
            if a[j-1] != b[i-1]:
                change += 1
            current_row[j] = min(add, delete, change)
    return current_row[n]

In [ ]:
for i in tq(askp):
    for j in sap:
        try:
            if distance(i[0], j[0]) < 2: 
                if (distance(i[1], j[1]) < 2 or i[1] in j[1] or j[1] in i[1]):
                    if (distance(i[2], j[2]) < 2 or i[2] in j[2] or j[2] in i[2]):
                        with open('result.txt', 'a') as f:
                            f.write('{}-----{}'.format(i,j)+'\n')
        except IndexError:
            with open('result.txt', 'a') as f:
                f.write('{}-----{}'.format(i,j)+'\n')

### Mapping family name with route

In [ ]:
file_sap = pd.read_csv('askp_route.csv', sep=';', encoding = 'utf-8', error_bad_lines=False, low_memory=False)

file_askp = pd.read_csv('sap_route.csv', sep=';', encoding = 'utf-8', error_bad_lines=False, low_memory=False)

In [ ]:
askp_route = file_askp.values.tolist()
sap_route = file_sap.values.tolist()

#### Preporation sap-set

In [ ]:
for i in tq(sap_route):
    for j in sap_route:
        if i[2].isspace():
            name_i = preproc(i[1])
            name_j = preproc(j[1])
            if distance(name_i[0], name_j[0]) < 2 and not j[2].isspace():
                try:
                    if distance(name_i[1], name_j[1]) < 2 and distance(name_i[2], name_j[2]) < 2: 
                        ind = sap_route.index(i)
                        sap_route[ind].append(j[2])
                    if ((len(name_i[1]) == 1 or len(name_j[1]) == 1) and name_i[1][0] == name_j[1][0]) and ((len(name_i[2]) == 1 or len(name_j[2]) == 1) and name_i[2][0] == name_j[2][0]):
                        ind = sap_route.index(i)
                        sap_route[ind].append(j[2])
                
                except IndexError:
                    ind = sap_route.index(i)
                    sap_route[ind].append(j[2])

#### Mapping two data-sets

In [ ]:
sap_buf =[i for i in sap_route]

In [ ]:
results = []
for i in sap_buf:
    flag = False
    for j in askp_route:
        if not i[2].isspace():
                if i[2] == j[2].strip():
                    name_i = preproc(i[1])
                    name_j = preproc(j[1])
                    if distance(name_i[0], name_j[0]) < 2:
                        try:
                            if distance(name_i[1], name_j[1]) < 3:
                                if distance(name_i[2], name_j[2]) < 4:
                                    results.append(i+j)
                                    print(results[-1])
                                    flag = True
                            if ((len(name_i[1]) == 1 or len(name_j[1]) == 1) and name_i[1][0] == name_j[1][0]):
                                if ((len(name_i[2]) == 1 or len(name_j[2]) == 1) and name_i[2][0] == name_j[2][0]):
                                    results.append(i+j)
                                    print(results[-1])
                                    flag = True
                
                        except IndexError:
                            results.append(i+j)
                            print(results[-1])
                            flag = True
        else:
            name_i = preproc(i[1])
            name_j = preproc(j[1])
            if distance(name_i[0], name_j[0]) < 2 and j[2] not in i[2:]:
                try:
                    if distance(name_i[1], name_j[1]) < 3:
                        if distance(name_i[2], name_j[2]) < 4:
                            routs = '/'.join(i[2:])
                            results.append(i[:2]+[routs]+j)
                            print(results[-1])
                            flag = True
                    if ((len(name_i[1]) == 1 or len(name_j[1]) == 1) and name_i[1][0] == name_j[1][0]):
                        if ((len(name_i[2]) == 1 or len(name_j[2]) == 1) and name_i[2][0] == name_j[2][0]):
                            routs = '/'.join(i[2:])
                            results.append(i[:2]+[routs]+j)
                            print(results[-1])
                            flag = True
                except IndexError:
                    routs = '/'.join(i[2:])
                    results.append(i[:2]+[routs]+j)
                    print(results[-1])
                    flag = True
    if not flag:
        name_i = preproc(i[1])
        name_j = preproc(j[1])
        if distance(name_i[0], name_j[0]) < 2:
            try:
                if distance(name_i[1], name_j[1]) < 3:
                    if distance(name_i[2], name_j[2]) < 4:
                        if len(i)>3:
                            routs = '/'.join(i[2:])
                            results.append(i[:2]+[routs]+j)
                            print(results[-1])
                        else:
                            results.append(i+j)
                            print(results[-1])
                        flag = True
                if ((len(name_i[1]) == 1 or len(name_j[1]) == 1) and name_i[1][0] == name_j[1][0]):
                    if ((len(name_i[2]) == 1 or len(name_j[2]) == 1) and name_i[2][0] == name_j[2][0]):
                        if len(i)>3:
                            routs = '/'.join(i[2:])
                            results.append(i[:2]+[routs]+j)
                            print(results[-1])
                        else:
                            results.append(i+j)
                            print(results[-1])
                        flag = True
            except IndexError:
                if len(i)>3:
                    routs = '/'.join(i[2:])
                    results.append(i[:2]+[routs]+j)
                    print(results[-1])
                else:
                    results.append(i+j)
                    print(results[-1])
                flag = True
    if not flag:
        if len(i)>3:
            routs = '/'.join(i[2:])
            results.append(i[:2]+[routs]+['None', 'None', 'None'])
            print(results[-1])
        else:
            results.append(i+['None', 'None', 'None'])
            print(results[-1])

In [ ]:
df = pd.DataFrame({
    'id sap':[i[0] for i in results],
    'name sap': [i[1] for i in results],
    'sap route': [i[2] for i in results],
    'id askp': [i[3] for i in results],
    'name askp': [i[4] for i in results],
    'askp route': [i[5] for i in results]
})

In [ ]:
writer = pd.ExcelWriter(r'output.xlsx')
df.to_excel(writer)
writer.save()

In [ ]:
askp_in = []
for i in results:
    if type(i[3]) is not str:
        askp_in.append(i[3])

In [ ]:
askp_exception = []
for i in askp_route:
    if i[0] not in askp_in:
        askp_exception.append(i)

In [ ]:
df1 = pd.DataFrame({
    'id askp':[i[0] for i in askp_exception],
    'name askp': [i[1] for i in askp_exception],
    'askp route': [i[2] for i in askp_exception]
})

writer = pd.ExcelWriter(r'output_askp.xlsx')
df1.to_excel(writer)
writer.save()